In [ ]:
import cv2
import numpy as np

In [ ]:
def on_EVENT_LBUTTONDOWN(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        xy = "%d,%d" % (x, y)
        a.append(x)
        b.append(y)
        cv2.circle(img, (x, y), 1, (0, 0, 255), thickness=-1)
        cv2.putText(img, xy, (x, y), cv2.FONT_HERSHEY_PLAIN,
                    1.0, (0, 0, 0), thickness=1)
        cv2.imshow("image", img)
        print(x,y)

In [ ]:
def normalizePoints(points):
    points = np.asarray(points)
    mean = np.mean(points,axis=0)
    points_x = points[:,0]
    points_y = points[:,1]
    std_x = np.std(points_x)
    std_y = np.std(points_y)
    std = np.sqrt((std_x**2+std_y**2)/2)
    T = np.array([[1/std,0,-mean[0]/std],
                  [0,1/std,-mean[1]/std],
                  [0,0,1]])
    homo_points = np.c_[points,np.ones([points.shape[0],1])]
    normalize_points = T.dot(homo_points.T)[0:2,:]
    return normalize_points.T, T

In [ ]:
img = cv2.imread('./000120.png')

a = []
b = []

cv2.namedWindow("image", cv2.WND_PROP_FULLSCREEN)
cv2.setMouseCallback("image", on_EVENT_LBUTTONDOWN)
cv2.imshow("image", img)
cv2.waitKey(0)
print(a[0], b[0])

In [ ]:
# Cone markers pixel coordinates on the image
pts_src = np.ones([int(len(a)),2])
for i in range(len(a)):
        pts_src[i,0] = a[i]
        pts_src[i,1] = b[i]

In [ ]:
# Cone markers pyhsical coordinates on the ground in reference to projected camera
# Please rearrange them in your condition
width = 1.27
height = 1.3081
pts_dst = np.array([[-width,0],
                    [0,0],
                    [width,0],
                    [2*width,0],
                    [-width, height],
                    [0, height],
                    [width, height],
                    [2*width, height],
                    [-2*width, 2*height],
                    [-width, 2*height],
                    [0, 2*height],
                    [width, 2*height],
                    [2*width, 2*height],
                    [-2*width, 3*height],
                    [-width, 3*height],
                    [0, 3*height],
                    [width, 3*height],
                    [2*width, 3*height],
                    [-2*width, 4*height],
                    [-width, 4*height],
                    [0, 4*height],
                    [width, 4*height],
                    [2*width, 4*height]])
camera_position = np.array([[-0.9144,4.4323]])
pts_dst += camera_position

In [ ]:
# Estiamte H matrix
N_pts_src, T_src = normalizePoints(pts_src)
N_pts_dst, T_dst = normalizePoints(pts_dst)
h, status = cv2.findHomography(N_pts_src, N_pts_dst, cv2.RANSAC, 5.0)

H = ((np.asmatrix(T_dst).I).dot(h)).dot(T_src)
H

In [ ]:
# check homography estimation error
for pt, gt in zip(pts_src, pts_dst):
    xy = np.array([pt[0], pt[1], 1]).reshape(3,1)
    a = H.dot(xy)
    b = a/a[-1]
    error = abs(b[0,0]-gt[0])+abs(b[1,0]-gt[1])
    print(error)